#Libraries and Data

In [3]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime
from ta import add_all_ta_features

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score

# Reporting
import matplotlib.pyplot as plt

In [4]:
def load_and_setup_data(sybmol):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
  #  df.set_index("Date", inplace=True)
   # new_data = pd.DataFrame(input_data)
   # new_data.set_index("Date", inplace=True)
    #concatenated_df = pd.concat([df,new_data])
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    return data
    

In [26]:
#symbols = ["CONCOR.NS","ELGIEQUIP.NS"]
symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI","RELI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","TITAN.NS","SUNPHARMA.BO","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]

#symbols = ["^NSEI","RELI"]

Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
#print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    data = load_and_setup_data(symbol)
    data = data[["Open", "High", "Low", "Close","Volume"]]
    data['weekday'] =pd.to_datetime(data.index).dayofweek
   # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
 
    # Find NaN Rows
    #na_list = data.columns[data.isna().any().tolist()]
   # data.drop(columns=na_list, inplace=True)
    # Handle inf values
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=1, inplace=True)
    #data = data.drop(['volatility_kchi','volatility_kcli'], axis=1)

    #df_stationary.head()
   # print(data)
    non_stationaries = []
    for col in data.columns:
        if col != "volatility_kchi" and col != "volatility_kcli" and col != "weekday":
            print(col)
            dftest = adfuller(data[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    
    df_stationary = data.copy()
    
 
  
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/indicator/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
   # print(feature_item,symbol,len(feature_item))
    #print("point zero :" ,len(df_stationary.columns))

    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    df_stationary = df_stationary.iloc[1:]
    # Find NaN Rows
    na_list = df_stationary.columns[df_stationary.isna().any().tolist()]
    
    df_stationary.drop(columns=na_list, inplace=True)
    print(feature_item)
    #print("point one :" ,len(df_stationary.columns))
    df_stationary = df_stationary[feature_item]
    #print("point two :" ,len(df_stationary.columns))

#    print(df_stationary)
    df_stationary = df_stationary.iloc[1:]
    
    df_stationary[np.isinf(df_stationary)] = np.nan  # Replace inf with NaN
    df_stationary = df_stationary.dropna()
    print("point three",len(df_stationary.columns),symbol)
    has_inf = "no inf data"
    if np.isinf(df_stationary).any().any():
        has_inf = "has inf data"
    df_stationary.replace(np.inf, np.nan, inplace=True)
    df_stationary.dropna(inplace=True)

        
    #print("check infifty value in",has_inf)
   # print(symbol," :",len(df_stationary.columns),len(feature_item))
    finaldata[symbol] = df_stationary 
    df_stationary = StandardScaler().fit_transform(df_stationary)
    y_pred = loaded_model.predict(df_stationary)
    y_pred = loaded_model.predict(df_stationary)

    #print("got result",symbol)
    # Specify Target

    #final training
    loaded_models[symbol] = loaded_model
    stock_name_only = stock_name.replace(".NS","")
   
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
    print(greater)
    final_result[symbol] = [current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)])

print(result)


[*********************100%***********************]  17 of 17 completed


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


             Open         High          Low        Close   Volume  weekday  \
0        6.956121     7.141618     6.770625     6.956121    62000        3   
1        6.956121     7.605359     6.956121     7.512611   188800        3   
2        7.512611     8.532841     6.538754     7.234366   201400        3   
3        6.770625     7.419863     6.724251     7.187992    74400        3   
4        7.651732     7.651732     7.234365     7.419862    42700        3   
...           ...          ...          ...          ...      ...      ...   
4573  1323.050049  1419.000000  1300.199951  1394.150024  6260935        3   
4574  1405.000000  1465.000000  1299.000000  1362.599976  3942116        3   
4575  1371.400024  1394.000000  1315.000000  1373.650024  2302335        3   
4576  1371.400024  1394.000000  1315.000000  1373.650024  2306059        3   
4577  1371.400024  1394.000000  1315.000000  1373.650024  2306059        3   

        volume_adi  volume_obv  volume_cmf     volume_fi  ...  

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


            Open        High         Low       Close  Volume  weekday  \
0      74.879914   74.883897   72.581617   73.206978   13884        3   
1      73.131302   73.844292   72.167373   72.306786   18075        3   
2      72.326716   73.680998   71.299054   72.302818  168093        3   
3      70.900720   71.697356   69.705765   71.342850  224990        3   
4      68.510799   71.689385   68.510799   70.024414   19715        3   
...          ...         ...         ...         ...     ...      ...   
4574  678.000000  683.900024  672.000000  683.650024  623895        3   
4575  683.250000  686.950012  675.099976  678.349976  707929        3   
4576  678.500000  690.000000  677.250000  688.799988  554084        3   
4577  678.500000  690.000000  677.250000  688.799988  554148        3   
4578  678.500000  690.000000  677.250000  688.799988  554148        3   

        volume_adi  volume_obv  volume_cmf     volume_fi  ...  momentum_ppo  \
0    -6.341473e+03       13884   -0.456747  

...

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


             Open         High          Low        Close   Volume  weekday  \
0      134.570376   135.357857   131.507980   131.954224    99480        3   
1      133.782930   136.495343   132.251732   135.948486   148210        3   
2      136.591558   137.895255   134.045392   137.256531   177430        3   
3      136.495329   137.807784   133.870418   134.920380   122270        3   
4      135.191609   140.432684   135.191609   137.768402   340780        3   
...           ...          ...          ...          ...      ...      ...   
3320  1000.950012  1025.199951   995.299988  1022.400024    64379        3   
3321   995.049988  1056.000000   995.049988  1051.400024  1169702        3   
3322  1039.949951  1051.000000  1028.349976  1035.500000    61772        3   
3323  1035.500000  1050.900024  1032.099976  1046.050049    27294        3   
3324  1077.050049  1079.349976  1064.849976  1072.400024    33392        3   

        volume_adi  volume_obv  volume_cmf     volume_fi  ...  

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


             Open         High          Low        Close   Volume  weekday  \
0      198.402787   205.608027   197.447399   203.617630  1268756        3   
1      205.807058   209.389773   204.831759   206.225037  1706608        3   
2      205.846880   208.792678   205.359242   207.240158   840360        3   
3      207.598396   207.996476   201.020137   202.682114   734988        3   
4      202.622417   209.549008   201.049998   208.016388  2080960        3   
...           ...          ...          ...          ...      ...      ...   
3336  1159.949951  1159.949951  1135.599976  1147.150024  2088505        3   
3337  1154.650024  1166.199951  1145.099976  1161.449951  1212566        3   
3338  1166.000000  1169.000000  1153.449951  1159.400024  1274203        3   
3339  1159.400024  1191.900024  1153.500000  1175.250000  3215610        3   
3340  1183.949951  1232.000000  1183.000000  1228.650024  6031344        3   

        volume_adi  volume_obv  volume_cmf     volume_fi  ...  

FileNotFoundError: [Errno 2] No such file or directory: '../stock_historical_data/ASIANPAINT.NS.csv'

In [27]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '15-07-2023',
   'result': '-1',
   'pre_close': '1373.6500244140625',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'OLECTRA',
   'prob': '60.87'},
  {'cdate': '15-07-2023',
   'result': '1',
   'pre_close': '688.7999877929688',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'CONCOR',
   'prob': '42.58'},
  {'cdate': '15-07-2023',
   'result': '-1',
   'pre_close': '572.9500122070312',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'ELGIEQUIP',
   'prob': '58.31'},
  {'cdate': '15-07-2023',
   'result': '-1',
   'pre_close': '96.5999984741211',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'IOC',
   'prob': '52.06'},
  {'cdate': '15-07-2023',
   'result': '-1',
   'pre_close': '127.1500015258789',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'BEL',
   'prob': '52.93'},
  {'cdate': '15-07-2023',
   'result': '-1',
   'pre_close': '7730.75',
   'isClassification': 1,
   'model': 'indi

In [28]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [ ]:
#final_target

In [ ]:
for symbol in symbols:
  
    # Set Target (for Supervised ML later on)
    finaldata[symbol]["TARGET"] = -1
    finaldata[symbol].loc[finaldata[symbol]["Close"].shift(-1) > finaldata[symbol]["Close"], "TARGET"] = 1
    finaldata[symbol].dropna(inplace=True)
    #print(finaldata)
    df_tar = finaldata[symbol][["TARGET"]]
    df_stationar = StandardScaler().fit_transform(finaldata[symbol].iloc[:, :-1])
   # print(df_tar)

    retrained_model = loaded_models[symbol].fit(df_stationar,df_tar)
    print(df_stationar)
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(retrained_model, f)
